
### Randomization Test Experiment 
* One of the key findings of the paper is that deep neural networks easily fit random noise. To validate this, we are training a neural network on a dataset where the labels are randomly assigned and evaluate its performance on the original test set.The random label test can be performed in two ways:
    
    * Complete label shuffle: The labels are shuffled completely and the network is trained on the shuffled labels. The network is then evaluated on the original test set.
    * Shuffled pixels : The pixels of all the images are permuted using a single random permutation and the network is trained on the shuffled dataset. The network is then evaluated on the original test set.
    * Random Pixels : The pixels of all the images are replaced with random values and the network is trained on the shuffled dataset. The network is then evaluated on the original test set.
    * Gaussian Noise : The pixels of all the images are replaced with random values drawn from a Gaussian distribution and the network is trained on the shuffled dataset. The network is then evaluated on the original test set.
    



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
from torchsummary import summary
from tqdm import tqdm
import tensorflow
import tensorflow as tf
from sklearn.metrics import accuracy_score



2023-04-22 01:01:37.943311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 01:01:39.683540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-22 01:01:39.683890: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-22 01:01:39.683905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameter
batch_size = 64
#  CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(28),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, transform=transform)

100%|██████████| 170498071/170498071 [09:02<00:00, 314440.87it/s] 


Extracting ./cifar-10-python.tar.gz to ./


In [5]:

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32, shuffle=True, num_workers=2)
test_loader =  torch.utils.data.DataLoader(test_dataset,batch_size=32, shuffle=False, num_workers=2)


# size of the dataset
print(train_loader.dataset.data.shape)
print(test_loader.dataset.data.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [3]:
def get_device():
  if torch.cuda.is_available():
      return torch.device('cuda')
  else:
      return torch.device('cpu')
device = get_device()
device



device(type='cpu')

In [4]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:
class ConvModule(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvModule, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out
    
class InceptionModule(nn.Module):
    def __init__(self, in_channels, Ch1x1 , Ch3x3):
        super(InceptionModule, self).__init__()
        self.branch1 = ConvModule(in_channels, Ch1x1, kernel_size=1, stride=1, padding=0)
        self.branch2 = ConvModule(in_channels, Ch3x3, kernel_size=3, stride=1, padding=1)
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        return torch.cat([branch1, branch2], 1)
    
class DownsampleModule(nn.Module):
    def __init__(self, in_channels, Ch3x3):
        super(DownsampleModule, self).__init__()
        self.branch1 = ConvModule(in_channels, Ch3x3, kernel_size=3, stride=2, padding=0)
        self.branch2 = nn.MaxPool2d(3, stride=2)
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        return torch.cat([branch1, branch2], 1)

class InceptionNet(nn.Module):
    def __init__(self, classes=10):
        super(InceptionNet, self).__init__()
        self.conv1 = ConvModule(in_channels=3, out_channels=96 , kernel_size=3, stride=1, padding=0)
        self.inception1 = InceptionModule(in_channels=96, Ch1x1=32 , Ch3x3=32)
        self.inception2 = InceptionModule(in_channels=64, Ch1x1=32 , Ch3x3=48)
        self.downsample1 = DownsampleModule(in_channels=80, Ch3x3=80)
        self.inception3 = InceptionModule(in_channels=160, Ch1x1=112 , Ch3x3=48)
        self.inception4 = InceptionModule(in_channels=160, Ch1x1=96 , Ch3x3=64)
        self.inception5 = InceptionModule(in_channels=160, Ch1x1=80 , Ch3x3=80)
        self.inception6 = InceptionModule(in_channels=160, Ch1x1=48 , Ch3x3=96)
        self.downsample2 = DownsampleModule(in_channels=144, Ch3x3=96)
        self.inception7 = InceptionModule(in_channels=240, Ch1x1=176 , Ch3x3=160)
        self.inception8 = InceptionModule(in_channels=336, Ch1x1=176 , Ch3x3=160)
        self.mean_pool = nn.AdaptiveAvgPool2d((7 , 7))
        self.fc = nn.Linear(16464, classes)
    def forward(self, x):
        out = self.conv1(x)
        out = self.inception1(out)
        out = self.inception2(out)
        out = self.downsample1(out)
        out = self.inception3(out)
        out = self.inception4(out)
        out = self.inception5(out)
        out = self.inception6(out)
        out = self.downsample2(out)
        out = self.inception7(out)
        out = self.inception8(out)
        out = self.mean_pool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [144]:

def genRandomlabeltrainloader(train_loader, randomLabel=False):
    if randomLabel:
        train_loader.dataset.targets = torch.randint(0, 10, (len(train_loader.dataset.targets),))
    return train_loader








def train(train_loader, test_loader ,epochs , randomLabel=False ,permutePixel = False ,randomPixel=False , GaussianNoise=False):
    model = InceptionNet().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    train_loss = []
    train_acclst = []
    perm = torch.randperm(3072)
    itr = 0
    if randomLabel:
        train_loader = genRandomlabeltrainloader(train_loader, randomLabel)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total = 0
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            # print(labels)
            # itr += 1
            # if randomLabel:
            #     torch.manual_seed(0)
            #     labels = labels[torch.randperm(len(labels))]
                # print(labels)
                # print("Assigned random labels")
            if randomPixel:
                images = torch.rand(images.size()).to(device)
            if permutePixel:
                images = images.view(-1, 75264)
                images = images[:, perm]
                images = images.view(-1, 64, 3, 32, 32)
            if GaussianNoise:
                images = (images + (torch.randn(images.size()) * 0.5).to(device)).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            total += labels.size(0)
            itr += 1
            if itr % 1000 == 0:
                acc_train  = accuracy_score(labels.cpu().detach().numpy(), torch.argmax(outputs.cpu().detach(), dim=1).numpy())
                print("Epoch: {} , Iteration: {} , Loss: {} , Accuracy: {}".format(epoch+1, itr, loss.item(), acc_train))
                train_acclst.append(acc_train)
        train_loss.append(running_loss/total)  
        scheduler.step()          
    return train_loss, model

In [145]:
Correcttrainloss , CorrectNet = train(train_loader, test_loader , 2 )


100%|██████████| 782/782 [16:37<00:00,  1.28s/it]


Epoch 0 Training Loss: 0.04187771620658728


100%|██████████| 782/782 [16:31<00:00,  1.27s/it]

Epoch 1 Training Loss: 0.03494754329514809


In [146]:
RandomLabeltrainloss , RandomLabelNet = train(train_loader, test_loader , 2 , randomLabel=True)


 58%|█████▊    | 454/782 [09:45<07:17,  1.33s/it]

In [ ]:
RandomPixeltrainloss , RandomPixelNet = train(train_loader, test_loader , 2 , randomPixel=True)


In [ ]:
PermutePixeltrainloss , PermutePixelNet = train(train_loader, test_loader , 2 , permutePixel=True)


In [ ]:
GaussianNoisetrainloss , GaussianNoiseNet = train(train_loader, test_loader , 2 , GaussianNoise=True)


In [12]:
InceptionSmallModel = InceptionNet()
summary(InceptionSmallModel, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 32, 32]           5,376
              ReLU-2          [-1, 192, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          12,352
              ReLU-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 96, 32, 32]          18,528
              ReLU-6           [-1, 96, 32, 32]               0
            Conv2d-7          [-1, 128, 32, 32]         110,720
              ReLU-8          [-1, 128, 32, 32]               0
            Conv2d-9           [-1, 16, 32, 32]           3,088
             ReLU-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 32, 32, 32]          12,832
             ReLU-12           [-1, 32, 32, 32]               0
        MaxPool2d-13          [-1, 192, 32, 32]               0
           Conv2d-14           [-1, 32,

In [ ]:

# plot train loss vs epoch
plt.plot(Correcttrainloss, label='Correct')
plt.plot(RandomLabeltrainloss, label='RandomLabel')
plt.plot(RandomPixeltrainloss, label='RandomPixel')
plt.plot(PermutePixeltrainloss, label='PermutePixel')
plt.plot(GaussianNoisetrainloss, label='GaussianNoise')
plt.legend()
plt.show()